In [1]:
import pandas as pd
import networkx as nx
import math as math

In [2]:
in_file = 'generated_data.tsv' # tsv of people with onboardID and companies
out_file = 'matches.tsv'       # tsv of matches generated, onboardID, company1, company2

manual_match_file = 'poamatches.tsv' # use if you have pre-assigned match data to include
# manual_match_file = false          #use if you don't have any pre-assigned matches to read in

## Read in people file
Should have an onboardID and columns of companies.

In [3]:
people = pd.read_csv(in_file, delimiter="\t", index_col='onboardID')
people.sample(5)

,acxiom,ancestry,beenverified,clubhouse,infinitemediaconcepts,intel,mcdonalds,mediaocean,mylife,name,...,quora,target,tesla,thomsonreuters,tmobile,twitter,uber,walgreens,walmart,xfinity
onboardID,,,,,,,,,,,,,,,,,,,,,
455,0,1,0,0,0,0,0,0,0,Amelie,...,0,1,0,0,0,0,0,1,1,0
555,0,0,0,0,0,0,0,0,0,Marjory,...,0,0,0,0,1,1,0,0,1,0
424,0,0,0,1,0,0,0,0,0,Jeannie,...,0,0,0,1,0,1,1,1,0,0
647,0,0,0,0,0,0,1,0,0,Brittnee,...,0,0,0,0,1,0,1,1,1,0
613,0,0,0,0,0,0,0,0,0,Violet,...,0,0,0,0,1,1,0,0,0,0


### Remove any manually-assigned groups, do filtering
Depending on how your input data was set up, you might want to remove the manually-assigned group.

In [4]:
# In our case, the POA group was manually-assigned. Remove them from the full data.
#people = people[people.isPOA == False]

## Read in the manually assigned group
They're pre-assigned! But we still need to include their companies so we can accurately count how many requests we're sending to each company overall.

This file should have at least an onboardID, company1, and company2. (You might have this data already in the people file too, so you could take that approach instead.)

In [5]:
if manual_match_file:
    manual_people = pd.read_csv(manual_match_file, delimiter="\t", index_col='onboardID')
    manual_people.sample(5)

In [6]:
# Sum the counts for each company that's already been assigned
manual_counts = {}
if manual_match_file:
    manual_counts = manual_people['company1'].append(manual_people['company2']).value_counts().to_dict()
    manual_counts

## Types of companies

In [7]:
company_type_dict = {
'acxiom':'broker',
'ancestry' :'consumer',
'beenverified': 'broker',
'clubhouse' : 'social',
'infinitemediaconcepts':'broker',
'intel': 'broker',
'mcdonalds' :'consumer',
'mediaocean': 'broker',
'mylife': 'broker',
'neustar' : 'broker',
'oracle' : 'broker',
'quora' : 'social',
'target' : 'consumer',
'tesla' : 'consumer',
'thomsonreuters':'broker',
'tmobile' : 'telco',
'twitter' : 'social',  
'uber': 'consumer',
'walgreens':'consumer',
'walmart':	'consumer',
'xfinity':	'telco',    
}

In [8]:
companies = company_type_dict.copy()

## Find the rarest comapnies

In [9]:
# Find the rarest companies.
rare_companies = people[list(companies.keys())].sum().sort_values()
rare_companies

beenverified              1
neustar                   1
mediaocean                1
mylife                    1
infinitemediaconcepts     2
tesla                     4
acxiom                    5
thomsonreuters            6
intel                     8
xfinity                  12
clubhouse                12
oracle                   13
ancestry                 14
mcdonalds                15
quora                    22
walmart                  28
target                   29
walgreens                32
tmobile                  38
uber                     38
twitter                  48
dtype: int64

### If less than 10% of people have an account, call it a rare company

In [10]:
# utility function
def isRare(company, numPeople=len(people.index)):
    return rare_companies[company] <= math.ceil(numPeople*.1)

### Make an empty matching matrix

In [11]:
match_matrix = people[list(companies.keys())].copy()
match_matrix = match_matrix.replace(match_matrix, 0)

## Network Flow Approach
Model this matching problem as a network flow problem with costs and capacity. The goal is to match up the most nodes with at the least cost. 

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.flow.max_flow_min_cost.html#networkx.algorithms.flow.max_flow_min_cost

In [12]:
g = nx.DiGraph()
nodes_people = list(people.index)
nodes_company = companies.keys()

# Define the maxes we're willing to send
MAX_REQUESTS = 200
PERSON_MAX = 2
COMPANY_MAX = math.ceil(len(nodes_people)*PERSON_MAX / len(nodes_company))

In [13]:
# The matrix that shows which people have accounts where
account_matrix = people[list(nodes_company)]

### Build nodes

In [14]:
# Build a directed graph with People nodes, Company nodes, a source, and a sink
g.add_nodes_from(nodes_people)
g.add_nodes_from(nodes_company)
g.add_node('s') # source node
g.add_node('t') # sink node

### Build the edges between nodes
Edges have a capacity and a cost. The cost is how 'expensive' it is to match a person to a company. 

Since we want more people to be matched with companies they have accounts with, we'll make those edges cheaper. If they don't have an account, the edge will be more expensive.

Edges with data brokers are free (cost=0), since no one has accounts with them anyway.
Edges with rare accounts give you a refund (cost=-1) since we want to incentivize those matches more.

(aside for another day: there is probably a better way to do this by incorporating a demand model as well.)

In [15]:
# Connect source and sink
for person, row in account_matrix.iterrows():
    g.add_edge('s', person, capacity=PERSON_MAX) #connect source to people
    
    for company, hasAccount in row.items():
        capacity = COMPANY_MAX
        if company in manual_counts: # reduce capacity for companies already manually assigned
            capacity = COMPANY_MAX - manual_counts[company] 
        g.add_edge(company, 't', capacity = capacity) #connect company to sink
        cost = 0
        if hasAccount:
            if isRare(company, len(nodes_people)):
                cost = -1
            else:
                cost = 0            
        elif company_type_dict[company] == 'broker':
            # no one is gonna have an account w data brokers, so don't punish them
            cost = 0
        else:
            # avoid pairing people who don't have accounts
            cost = 1 
        g.add_edge(person, company, capacity=1, weight=cost)

### Find matches!
That is, calculate the maximum total flow at the least cost.

In [16]:
matches = nx.algorithms.flow.max_flow_min_cost(g, 's', 't', capacity='capacity', weight='weight')
matches

{949: {'acxiom': 0,
  'ancestry': 0,
  'beenverified': 0,
  'clubhouse': 0,
  'infinitemediaconcepts': 0,
  'intel': 0,
  'mcdonalds': 0,
  'mediaocean': 0,
  'mylife': 0,
  'neustar': 0,
  'oracle': 0,
  'quora': 0,
  'target': 0,
  'tesla': 0,
  'thomsonreuters': 0,
  'tmobile': 1,
  'twitter': 1,
  'uber': 0,
  'walgreens': 0,
  'walmart': 0,
  'xfinity': 0},
 733: {'acxiom': 0,
  'ancestry': 0,
  'beenverified': 0,
  'clubhouse': 0,
  'infinitemediaconcepts': 0,
  'intel': 0,
  'mcdonalds': 1,
  'mediaocean': 0,
  'mylife': 0,
  'neustar': 0,
  'oracle': 0,
  'quora': 0,
  'target': 0,
  'tesla': 0,
  'thomsonreuters': 0,
  'tmobile': 0,
  'twitter': 0,
  'uber': 0,
  'walgreens': 0,
  'walmart': 1,
  'xfinity': 0},
 297: {'acxiom': 0,
  'ancestry': 1,
  'beenverified': 0,
  'clubhouse': 0,
  'infinitemediaconcepts': 0,
  'intel': 0,
  'mcdonalds': 0,
  'mediaocean': 0,
  'mylife': 0,
  'neustar': 0,
  'oracle': 0,
  'quora': 0,
  'target': 0,
  'tesla': 0,
  'thomsonreuters': 1,
 

In [17]:
# calculate the cost just as FYIs/sanity check. Should be close to 0.
# Negative means there's more matches-with-account than matches-without-account
nx.algorithms.flow.cost_of_flow(g, matches) 

-22

### Build the Person-Company match matrix
The flow algorithms spit out residual graph dictionaries, which are a pain to work with. Convert this to a nice dataframe instead.

1 means match. 0 means no match.

In [18]:
match_matrix = people[list(nodes_company)].copy()
match_matrix = match_matrix.replace(match_matrix, 0)


for company in nodes_company:
    for oid in nodes_people:
        match_matrix.at[oid, company] = matches[oid][company]
        
match_matrix.head()

,acxiom,ancestry,beenverified,clubhouse,infinitemediaconcepts,intel,mcdonalds,mediaocean,mylife,neustar,...,quora,target,tesla,thomsonreuters,tmobile,twitter,uber,walgreens,walmart,xfinity
onboardID,,,,,,,,,,,,,,,,,,,,,
949,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
733,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
297,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
930,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
938,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


## How good was our matching?

In [19]:
matched_accounts = match_matrix & account_matrix 
matched_noaccount = match_matrix &  (match_matrix ^ account_matrix)

frame = {"w account" : matched_accounts.sum(),
         "w/out account" : matched_noaccount.sum(),
         "total": matched_accounts.sum() + matched_noaccount.sum()
        }
display("Total Matches by Company. (Note this doesn't include manually matched cases.)")
pd.DataFrame(frame).sort_values(by=["total", "w/out account", "w account"])

"Total Matches by Company. (Note this doesn't include manually matched cases.)"

,total,w account,w/out account
tesla,7,4,3
beenverified,8,1,7
mediaocean,8,1,7
mylife,8,1,7
oracle,8,1,7
clubhouse,9,9,0
quora,9,9,0
twitter,9,9,0
xfinity,9,9,0
acxiom,9,5,4


In [20]:
# include the manually-matched group this time, and let's look at the final numbers
display("Total Matches by Company. (Includes the imported manually matched cases.)")
match_totals = pd.DataFrame(frame['total'].add(pd.Series(manual_counts, dtype=int), fill_value=0)).astype(int)
match_totals = match_totals.rename(columns={0:"total DARs"})
match_totals

'Total Matches by Company. (Includes the imported manually matched cases.)'

,total DARs
acxiom,11
ancestry,12
beenverified,10
clubhouse,10
infinitemediaconcepts,13
intel,11
mcdonalds,13
mediaocean,10
mylife,10
neustar,10


In [21]:
# Example for if you want to see who is assigned to Intel
#account_matrix['intel'] ==1

# Example for if you want to see what companies a person is assigned to
#account_matrix.loc[56] == 1

# Example for if you want to manually rebalance. e.g., switch 55 from neustar to quora
#match_matrix.at[55, "neustar"]=0
#match_matrix.at[55, "quora"]=1

## Summarize

In [22]:
print("Number of people:", len(people.index)+len(manual_people.index))
print("Numer of companies: ", len(companies))
print("DARs/company: ")
match_totals.apply([min, max, sum])

Number of people: 113
Numer of companies:  21
DARs/company: 


,total DARs
min,8
max,13
sum,226


## Put the matches in a handy format for exporting

In [23]:
export_matches = pd.DataFrame(index = nodes_people, columns=["company1", "company2"])
export_matches.index = export_matches.index.rename('onboardID')
for person in nodes_people:
    cos = [company for company, isMatched in match_matrix.loc[person].to_dict().items() if isMatched]
    export_matches.at[person, 'company1'] = cos[0]
    export_matches.at[person, 'company2'] = cos[1]
    
export_matches.sample(4)

,company1,company2
onboardID,,
980,ancestry,walmart
576,tesla,twitter
297,ancestry,thomsonreuters
351,beenverified,target


In [24]:
export_matches.to_csv(out_file, sep='\t')